In [33]:
import Numerical_Stratification 

import importlib 

importlib.reload(Numerical_Stratification) 

<module 'Numerical_Stratification' from 'c:\\Users\\Adem.ait\\OneDrive - University of Luxembourg\\Feina\\PhD\\sampling\\sample-creator\\Numerical_Stratification.py'>

In [35]:
# MAIN CODE - PREPROCESSING
print("PREPROCESSING:")
file_path = r"type_likes.csv"
df = Numerical_Stratification.read_dataframe(file_path)
print(df)
df = df.drop("type", axis=1)
df_clean = Numerical_Stratification.analyze_df(df)
variables = Numerical_Stratification.create_lists_from_df(df_clean) #diccionario de las variables -> lista de valores
keys, values = Numerical_Stratification.dictionary_to_lists(variables)
statistics = Numerical_Stratification.print_and_collect_statistics(variables)

PREPROCESSING:
                                     name     type  likes
0                  0-hero/OIG-small-chip2  dataset      8
1                   0-hero/prompt-perfect  dataset      0
2                        000alen/semantic  dataset      0
3                      000hen/captchaCode  dataset      0
4                        003myjoker1/test  dataset      0
...                                   ...      ...    ...
674822               zzzzxx/5656ChatPaper    space      0
674823                   zzzzxx/ChatPaper    space      0
674824                  zzzzzz/text2image    space      1
674825               zzzzzz567/Real-CUGAN    space      0
674826  zzzzzz567/vits-uma-genshin-honkai    space      0

[674827 rows x 3 columns]
The dataframe does not contain NaN values.
Statistics for numerical variable 'likes':
  Population size: 674827
  Mean: 1.134440382498033
  Median: 0.0
  Standard Deviation: 28.127517598840843



In [36]:
# MAIN CODE - STRATIFICATION
# elbow_method(values, 5)
print("STRATIFICATION:")
num_clusters_list = [3, 3] # Specify the number of clusters for each variable
strata = Numerical_Stratification.create_stratum_kmeans(variables, num_clusters_list) # Apply KMeans clustering to each variable
Numerical_Stratification.print_stratum_counts(strata) #Print the number of elements in each stratum for each variable

STRATIFICATION:
Variable: likes
  Stratum 1: 674493 points
  Stratum 2: 21 points
  Stratum 3: 313 points


In [37]:
# MAIN CODE - COMBINATION
print("COMBINATION:")
stratum_ranges = Numerical_Stratification.get_stratum_ranges(strata)
print("Ranges:", stratum_ranges)
strata_combinations = Numerical_Stratification.combination(stratum_ranges) 
print("Combinations of ranges:") 
for comb in strata_combinations:
    print(comb)
print(len(strata_combinations))
observations = Numerical_Stratification.df_to_list_observations(df_clean) # SE LLAMARÁ DESDE CLASSIFY_OBSERVATIONS
print(len(observations))

classified_observations_before = Numerical_Stratification.classify_observations(observations, strata_combinations)

# Print the number of observations in each stratum in the order of defined strata
print("\nBefore dropping empty strata:")
Numerical_Stratification.print_combination_stratum_counts(classified_observations_before, strata_combinations)
# Make a copy of classified observations
classified_observations_after = classified_observations_before.copy()
# Drop stratum with 0 observations
Numerical_Stratification.drop_empty_strata(classified_observations_after, strata_combinations)
# Print the number of observations in each stratum in the order of defined strata after dropping empty strata
print("\nAfter dropping empty strata:")
Numerical_Stratification.print_combination_stratum_counts(classified_observations_after, strata_combinations)

COMBINATION:
Ranges: {'likes': [[0, 315], [2285, 9909], [317, 1930]]}
Combinations of ranges:
[[0, 315]]
[[2285, 9909]]
[[317, 1930]]
3
674827

Before dropping empty strata:
Stratum [[0, 315]]: 674493 observations
Stratum [[2285, 9909]]: 21 observations
Stratum [[317, 1930]]: 313 observations

Total sum of observations in all strata: 674827

After dropping empty strata:
Stratum [[0, 315]]: 674493 observations
Stratum [[2285, 9909]]: 21 observations
Stratum [[317, 1930]]: 313 observations

Total sum of observations in all strata: 674827


In [38]:
# MAIN CODE - PRESAMPLING
print("PRE-SAMPLING")
N, mu = Numerical_Stratification.extract_population_size_and_means(statistics)
print("Population size (N):", N)
print("Means (mu):", mu)
nis, phi = Numerical_Stratification.nis_phi(classified_observations_after, N) # Calculate the stratum proportions
s = Numerical_Stratification.calculate_variable_std_devs(classified_observations_after) # Calculate the standard deviations for each stratum
print("nis:", len(nis), nis)
print("phi:", len(phi), phi)
print("s: ", s)
print("s:")
for variable_index, std_devs_for_variable in enumerate(s, start=1):
    print(f"  Variable {variable_index}: {std_devs_for_variable}")

sample_sizes, strata = Numerical_Stratification.calculate_sample_sizes(mu, 0.95, phi, s, 2, N, nis) # Call the function to calculate sample sizes
print("Size of the global sample:", sample_sizes)
print("Sample sizes for each stratum:", strata)

max_n, max_n_dist, max_n_idx = Numerical_Stratification.get_max_sample_distribution(sample_sizes, strata)
print("Variable wiht Max n: ", max_n, max_n_dist)

filtered_max_n_dist, filtered_phi, filtered_nis, filtered_s, filtered_classified_observations = Numerical_Stratification.filter_zero_strata(
    max_n_dist=max_n_dist,
    phi=phi,
    nis=nis,
    s=s,
    max_n_idx=max_n_idx,
    classified_observations=classified_observations_after
)

print("Filtered max_n_dist:", len(filtered_max_n_dist), filtered_max_n_dist)
print("Filtered phi:", len(filtered_phi), filtered_phi)
print("Filtered nis:", len(filtered_nis), filtered_nis)
print("Filtered s:", len(filtered_s), filtered_s)
print("Filtered classified_observations:", len(filtered_classified_observations))

PRE-SAMPLING
Population size (N): 674827
Means (mu): [1.134440382498033]
nis: 3 [674493, 21, 313]
phi: 3 [0.9995050583334691, 3.111908681780664e-05, 0.0004638225797130227]
s:  [[7.216813357658444, 1781.2013888103727, 351.4946950794105]]
s:
  Variable 1: [7.216813357658444, 1781.2013888103727, 351.4946950794105]
Size of the global sample: [56879]
Sample sizes for each stratum: [[56851, 2, 26]]
Variable wiht Max n:  56879 [56851, 2, 26]
Filtered max_n_dist: 3 [56851, 2, 26]
Filtered phi: 3 [0.9995050583334691, 3.111908681780664e-05, 0.0004638225797130227]
Filtered nis: 3 [674493, 21, 313]
Filtered s: 1 [[7.216813357658444, 1781.2013888103727, 351.4946950794105]]
Filtered classified_observations: 3


In [27]:

# MAIN - SAMPLING 
# Assuming `filtered_classified_observations`, `filtered_phi`, `filtered_nis`, `filtered_s`, `max_n_idx`, `max_n`, and `filtered_max_n_dist` are defined in the context of your program
Numerical_Stratification.automated_sampling(filtered_classified_observations, filtered_phi, filtered_nis, filtered_s, max_n_idx, max_n, filtered_max_n_dist)

NameError: name 'variables' is not defined